# A* Pathfinding on a Weighted Maze
We assign movement costs to a grid and use the A* algorithm with the Manhattan heuristic to find the cheapest path.

### Understanding the A* Algorithm

A* (pronounced "A-star") is a pathfinding and graph traversal algorithm that is widely used for finding the shortest path between two points, called nodes. It is an informed search algorithm, as it uses a heuristic function to guide its search towards the goal.

#### Key Concepts

The core of A* is the formula:
**`f(n) = g(n) + h(n)`**

Where:
- **`n`** is the next node on the path.
- **`g(n)`** is the **actual cost** of the path from the start node to `n`. In our maze, this is the sum of the costs of all cells we have traversed to get to `n`.
- **`h(n)`** is the **heuristic (estimated) cost** from `n` to the goal. This is an educated guess. The quality of this guess is crucial for A*'s performance. For the heuristic to be optimal, it must be **admissible**, meaning it never overestimates the actual cost.
- **`f(n)`** is the total estimated cost of the path through `n`. A* always explores the node with the lowest `f(n)` value first.

#### How It Works

1.  **Initialization**: A* starts with a priority queue (often called the "frontier" or "open set") containing the starting node. The priority is determined by the `f(n)` value.
2.  **Iteration**: The algorithm repeatedly extracts the node with the lowest `f(n)` from the priority queue.
3.  **Goal Test**: If the extracted node is the goal, the search is complete.
4.  **Expansion**: If it's not the goal, the algorithm finds all its neighbors and calculates their tentative `g(n)` and `f(n)` scores.
5.  **Update Frontier**: If a neighbor is reached via a cheaper path than before, its scores are updated, and it's added to the priority queue.

This process guarantees that the first time we reach the goal, it will be through the cheapest possible path.

In [1]:
from heapq import heappush, heappop

# None marks a blocked cell, numbers mark the cost to step into that cell
cost_grid = [
    [1, 1, 2, None, 1],
    [1, None, 2, None, 2],
    [1, 1, 1, 1, 2],
    [2, None, None, 1, 1],
    [1, 1, 1, None, 1]
]

start = (0, 0)
goal = (4, 4)

def in_bounds(cell):
    r, c = cell
    return 0 <= r < len(cost_grid) and 0 <= c < len(cost_grid[0])

def passable(cell):
    r, c = cell
    return cost_grid[r][c] is not None

def neighbors(cell):
    r, c = cell
    for dr, dc in ((1, 0), (-1, 0), (0, 1), (0, -1)):
        nxt = (r + dr, c + dc)
        if in_bounds(nxt) and passable(nxt):
            yield nxt

### Code: Grid Setup and Helper Functions

This cell sets up the environment for our A* search.

- **`cost_grid`**: This 2D list represents our maze.
    - `1`, `2`, etc., are the costs to move into that cell.
    - `None` represents a wall or an impassable obstacle.
- **`start`** and **`goal`**: These tuples define the starting and ending coordinates `(row, col)`.
- **`in_bounds(cell)`**: A utility function to check if a given cell's coordinates are within the grid's boundaries. This prevents the algorithm from trying to access invalid locations.
- **`passable(cell)`**: Checks if a cell is not a wall (`None`).
- **`neighbors(cell)`**: A generator function that yields all valid, passable neighbors of a given cell. It checks up, down, left, and right.

In [2]:
def manhattan(a, b):
    """Heuristic: distance if we can only move in straight lines."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

### Code: The Heuristic Function

The `manhattan` distance is our chosen heuristic function `h(n)`.

- **Why it works**: It calculates the distance between two points by summing the absolute differences of their coordinates. This is like moving on a grid, only horizontally and vertically.
- **Admissibility**: The Manhattan distance is admissible because it never overestimates the true cost. In a grid where the minimum movement cost is 1, the shortest path will always be at least the Manhattan distance. This property is essential for A* to find the optimal path.

In [3]:
def a_star(grid, start, goal):
    """Classic A* search that combines real cost (g) and heuristic (h)."""
    frontier = []
    heappush(frontier, (manhattan(start, goal), 0, start))  # (f_score, g_score, node)

    came_from = {start: None}
    g_score = {start: 0}

    while frontier:
        f, current_cost, current = heappop(frontier)
        if current == goal:
            break

        for nxt in neighbors(current):
            step_cost = grid[nxt[0]][nxt[1]]
            tentative_g = current_cost + step_cost

            if tentative_g < g_score.get(nxt, float("inf")):
                came_from[nxt] = current
                g_score[nxt] = tentative_g
                new_f = tentative_g + manhattan(nxt, goal)
                heappush(frontier, (new_f, tentative_g, nxt))

    return came_from, g_score

### Code: The A* Implementation

This is the main A* function, bringing all the concepts together.

- **`frontier`**: A priority queue (implemented using `heapq`) that stores nodes to be explored. It is ordered by the `f_score`, ensuring we always process the most promising node next. Each item in the queue is a tuple `(f_score, g_score, node)`.
- **`came_from`**: A dictionary that acts like a map, storing the path. For each node, it records the node that came immediately before it on the cheapest path found so far. This is used later to reconstruct the final path.
- **`g_score`**: A dictionary that stores the `g_score` (actual cost from the start) for each visited node.

#### The Loop

The `while frontier:` loop is the heart of the algorithm.
1.  It pops the node with the lowest `f_score` from the `frontier`.
2.  If this node is the `goal`, we've found the shortest path, and the loop breaks.
3.  Otherwise, it iterates through the `neighbors` of the current node.
4.  For each neighbor, it calculates a `tentative_g` score (the cost to reach the neighbor through the current node).
5.  If this `tentative_g` is better (lower) than any previously recorded `g_score` for that neighbor, it means we've found a cheaper path to the neighbor.
6.  We then update `came_from` to record this new path, update the neighbor's `g_score`, calculate its new `f_score`, and push it onto the `frontier`.

In [4]:
def build_path(came_from, start, goal):
    if goal not in came_from:
        return []
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path

### Code: Path Reconstruction

Once the `a_star` function has found the goal, the `came_from` dictionary contains all the information needed to find the path.

This function works by backtracking from the `goal`:
1. It starts with the `goal` node.
2. It looks up the `goal` in `came_from` to find the node that came before it.
3. It continues this process, following the chain of predecessors, until it reaches the `start` node (whose predecessor is `None`).
4. The path is then reversed to get the correct sequence from start to goal.

In [5]:
came_from, g_scores = a_star(cost_grid, start, goal)
path = build_path(came_from, start, goal)

if path:
    total_cost = g_scores[goal]
    print("Optimal path coordinates:")
    for step, cell in enumerate(path, start=1):
        print(f"{step}. {cell}")
    print("\nTotal path cost:", total_cost)
else:
    print("No path found.")

# Draw the grid with the path highlighted
visual_grid = []
for r, row in enumerate(cost_grid):
    visual_row = []
    for c, value in enumerate(row):
        if (r, c) == start:
            visual_row.append("S")
        elif (r, c) == goal:
            visual_row.append("G")
        elif (r, c) in path:
            visual_row.append("*")
        elif value is None:
            visual_row.append("#")
        else:
            visual_row.append(str(value))
    visual_grid.append(visual_row)

print("\nGrid view (S=start, G=goal, *=path, #=wall, numbers=cost):")
for row in visual_grid:
    print(" ".join(row))

Optimal path coordinates:
1. (0, 0)
2. (1, 0)
3. (2, 0)
4. (2, 1)
5. (2, 2)
6. (2, 3)
7. (3, 3)
8. (3, 4)
9. (4, 4)

Total path cost: 8

Grid view (S=start, G=goal, *=path, #=wall, numbers=cost):
S 1 2 # 1
* # 2 # 2
* * * * 2
2 # # * *
1 1 1 # G


### Conclusion

The final cell executes the A* search and visualizes the result.

- It calls `a_star` to get the path map (`came_from`) and costs (`g_scores`).
- `build_path` reconstructs the optimal path from the map.
- The total cost is retrieved from `g_scores[goal]`.
- Finally, a simple grid is printed to the console, where:
    - `S` is the Start
    - `G` is the Goal
    - `*` is the optimal path
    - `#` is a wall
    - Numbers are the original costs of the cells.

This provides a clear, visual confirmation of the path found by the algorithm.